In [1]:
import minari
import d3rlpy
import numpy as np
import os

# Parameters of the experiments

In [2]:
# Total number of training updates
n_steps = 1_000 # 100_000

# Number of updates between two evaluations (epochs)
n_steps_per_epoch = 100 # 1000

# Number of episodes during the testing phase
N = 50

# Delay between one step of the episode and the next in simulation
delay = 0.01

# Loading and preparation of datasets and environments

In [3]:
# Loading Minari datasets for the tasks
pen_dataset = minari.load_dataset("D4RL/pen/expert-v2")
relocate_dataset = minari.load_dataset("D4RL/relocate/expert-v2")
hammer_dataset = minari.load_dataset("D4RL/hammer/expert-v2")
door_dataset = minari.load_dataset("D4RL/door/expert-v2")

pen_env = pen_dataset.recover_environment()
relocate_env = relocate_dataset.recover_environment()
hammer_env = hammer_dataset.recover_environment()
door_env = door_dataset.recover_environment()

In [4]:
def prepare_d3_dataset(minari_dataset):
    # Lists to collect observations, actions, rewards, and terminals from all episodes
    observations = []
    actions = []
    rewards = []
    terminals = []

    # Iterate over episodes in the Minari dataset
    for episode in minari_dataset.iterate_episodes():
        # Extract sequences of data, removing the last observation
        obs = episode.observations[:-1]
        actions_ep = episode.actions
        rewards_ep = episode.rewards
        dones = np.array(episode.terminations) | np.array(episode.truncations)

        observations.append(obs)
        actions.append(actions_ep)
        rewards.append(rewards_ep)
        terminals.append(dones)

    # Merge all episodes into single arrays
    observations = np.concatenate(observations)
    actions = np.concatenate(actions)
    rewards = np.concatenate(rewards)
    terminals = np.concatenate(terminals)

    # Build the final dataset in d3rlpy's MDPDataset format
    d3_dataset = d3rlpy.datasets.MDPDataset(
        observations=observations,
        actions=actions,
        rewards=rewards,
        terminals=terminals,
        action_space=d3rlpy.constants.ActionSpace.CONTINUOUS
    )

    return d3_dataset

In [5]:
# Conversion of Minari datasets into MDPDataset format for training
pen_d3_dataset = prepare_d3_dataset(pen_dataset)
relocate_d3_dataset = prepare_d3_dataset(relocate_dataset)
hammer_d3_dataset = prepare_d3_dataset(hammer_dataset)
door_d3_dataset = prepare_d3_dataset(door_dataset)

2025-05-18 18:37.09 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('float32')], shape=[(24,)]) observation_signature=Signature(dtype=[dtype('float64')], shape=[(45,)]) reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)])
2025-05-18 18:37.09 [info     ] Action size has been automatically determined. action_size=24
2025-05-18 18:37.12 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('float32')], shape=[(30,)]) observation_signature=Signature(dtype=[dtype('float64')], shape=[(39,)]) reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)])
2025-05-18 18:37.12 [info     ] Action size has been automatically determined. action_size=30
2025-05-18 18:37.15 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('float32')], shape=[(26,)]) observation_signature=Signature(dtype=[dtype('float64')], shape=[(46,)]) reward_signature=S

# Training Algorithm

In [6]:
def train_offline_algorithm(config_class, dataset, env, filename, task):
    # Initialize the algorithm on CPU
    algo = config_class().create(device="cpu")

    # Build the neural networks based on the dataset
    algo.build_with_dataset(dataset)

    # Train the algorithm on the offline dataset and periodically evaluate online, saving the training history
    algo.fit(
        dataset=dataset,
        n_steps=n_steps,
        n_steps_per_epoch=n_steps_per_epoch,
        evaluators={"environment": d3rlpy.metrics.EnvironmentEvaluator(env)},
        logger_adapter=d3rlpy.logging.FileAdapterFactory(root_dir=f"training_logs/offline/{task}"),
    )

    algo.save(f'policies/offline/{filename}.d3')

# Creation of folders for policies and logs

In [8]:
# Create policies/offline
policies_path = os.path.join("policies", "offline")
if not os.path.exists(policies_path):
    os.makedirs(policies_path)
    print(f"Created: {policies_path}")
else:
    print(f"Already exists: {policies_path}")

# Create training_logs/offline/{task}
training_base = os.path.join("training_logs", "offline")
task_dirs = ["pen", "relocate", "hammer", "door"]

for task in task_dirs:
    task_path = os.path.join(training_base, task)
    if not os.path.exists(task_path):
        os.makedirs(task_path)
        print(f"Created: {task_path}")
    else:
        print(f"Already exists: {task_path}")

Created: policies/offline
Created: training_logs/offline/pen
Created: training_logs/offline/relocate
Created: training_logs/offline/hammer
Created: training_logs/offline/door


# Policy training

### Pen

In [9]:
train_offline_algorithm(d3rlpy.algos.IQLConfig, pen_d3_dataset, pen_env, 'pen_iql', 'pen')
train_offline_algorithm(d3rlpy.algos.CQLConfig, pen_d3_dataset, pen_env, 'pen_cql', 'pen')
train_offline_algorithm(d3rlpy.algos.BCConfig, pen_d3_dataset, pen_env, 'pen_bc', 'pen')
train_offline_algorithm(d3rlpy.algos.TD3PlusBCConfig, pen_d3_dataset, pen_env, 'pen_td3bc', 'pen')
train_offline_algorithm(d3rlpy.algos.AWACConfig, pen_d3_dataset, pen_env, 'pen_awac', 'pen')

2025-05-18 18:37.55 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float64')], shape=[(45,)]), action_signature=Signature(dtype=[dtype('float32')], shape=[(24,)]), reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), action_space=<ActionSpace.CONTINUOUS: 1>, action_size=24)
2025-05-18 18:37.55 [warning  ] Skip building models since they're already built.
2025-05-18 18:37.55 [info     ] Directory is created at training_logs/offline/pen/IQL_20250518183755
2025-05-18 18:37.55 [info     ] Parameters                     params={'observation_shape': [45], 'action_size': 24, 'config': {'type': 'iql', 'params': {'batch_size': 256, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'compile_graph': False, 'actor_learning_rate': 0.0003, 'critic_learning_rate': 0.0003, 'actor_optim_factory': {'type': '

Epoch 1/10:   0%|          | 0/100 [00:00<?, ?it/s]

2025-05-18 18:37.56 [info     ] IQL_20250518183755: epoch=1 step=100 epoch=1 metrics={'time_sample_batch': 0.0023915457725524904, 'time_algorithm_update': 0.006236391067504883, 'critic_loss': 2401.4978924560546, 'q_loss': 2401.4966394042967, 'v_loss': 0.0012520333076827229, 'actor_loss': 31.78553134918213, 'time_step': 0.008673124313354492, 'environment': 489.2528916382565} step=100
2025-05-18 18:37.56 [info     ] Model parameters are saved to training_logs/offline/pen/IQL_20250518183755/model_100.d3


Epoch 2/10:   0%|          | 0/100 [00:00<?, ?it/s]

2025-05-18 18:37.57 [info     ] IQL_20250518183755: epoch=2 step=200 epoch=2 metrics={'time_sample_batch': 0.0017271947860717774, 'time_algorithm_update': 0.007433810234069824, 'critic_loss': 693.5073046875, 'q_loss': 693.4424584960938, 'v_loss': 0.06484254992567003, 'actor_loss': 13.433249387741089, 'time_step': 0.009196622371673584, 'environment': 562.4906482424059} step=200
2025-05-18 18:37.57 [info     ] Model parameters are saved to training_logs/offline/pen/IQL_20250518183755/model_200.d3


...

Epoch 9/10:   0%|          | 0/100 [00:00<?, ?it/s]

2025-05-18 18:39.17 [info     ] AWAC_20250518183856: epoch=9 step=900 epoch=9 metrics={'time_sample_batch': 0.006782932281494141, 'time_algorithm_update': 0.010878560543060302, 'critic_loss': 402.0726000976563, 'actor_loss': 327483.6275, 'temp': 0.0, 'temp_loss': 0.0, 'time_step': 0.01769963502883911, 'environment': 35.59195899254889} step=900
2025-05-18 18:39.17 [info     ] Model parameters are saved to training_logs/offline/pen/AWAC_20250518183856/model_900.d3


Epoch 10/10:   0%|          | 0/100 [00:00<?, ?it/s]

2025-05-18 18:39.19 [info     ] AWAC_20250518183856: epoch=10 step=1000 epoch=10 metrics={'time_sample_batch': 0.006812191009521485, 'time_algorithm_update': 0.01053680419921875, 'critic_loss': 393.3514407348633, 'actor_loss': 316696.3978125, 'temp': 0.0, 'temp_loss': 0.0, 'time_step': 0.017386393547058107, 'environment': 205.61118658606387} step=1000
2025-05-18 18:39.19 [info     ] Model parameters are saved to training_logs/offline/pen/AWAC_20250518183856/model_1000.d3


(<d3rlpy.algos.qlearning.awac.AWAC at 0x162157d00>,
 [(1,
   {'time_sample_batch': 0.006597633361816406,
    'time_algorithm_update': 0.011078934669494628,
    'critic_loss': 1515.9315173339844,
    'actor_loss': 804936.3421875,
    'temp': 0.0,
    'temp_loss': 0.0,
    'time_step': 0.017715644836425782,
    'environment': 8.410851699632278}),
  (2,
   {'time_sample_batch': 0.006958942413330078,
    'time_algorithm_update': 0.012434933185577392,
    'critic_loss': 669.8427728271485,
    'actor_loss': 396145.97625,
    'temp': 0.0,
    'temp_loss': 0.0,
    'time_step': 0.01943585157394409,
    'environment': 748.8670424597892}),
  (3,
   {'time_sample_batch': 0.006732895374298096,
    'time_algorithm_update': 0.01150679111480713,
    'critic_loss': 546.1485437011719,
    'actor_loss': 398595.64734375,
    'temp': 0.0,
    'temp_loss': 0.0,
    'time_step': 0.018281190395355223,
    'environment': 553.0900370340382}),
  (4,
   {'time_sample_batch': 0.009654803276062012,
    'time_algor

### Relocate

In [10]:
train_offline_algorithm(d3rlpy.algos.IQLConfig, relocate_d3_dataset, relocate_env, 'relocate_iql', 'relocate')
train_offline_algorithm(d3rlpy.algos.CQLConfig, relocate_d3_dataset, relocate_env, 'relocate_cql', 'relocate')
train_offline_algorithm(d3rlpy.algos.BCConfig, relocate_d3_dataset, relocate_env, 'relocate_bc', 'relocate')
train_offline_algorithm(d3rlpy.algos.TD3PlusBCConfig, relocate_d3_dataset, relocate_env, 'relocate_td3bc', 'relocate')
train_offline_algorithm(d3rlpy.algos.AWACConfig, relocate_d3_dataset, relocate_env, 'relocate_awac', 'relocate')

2025-05-18 18:40.15 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float64')], shape=[(39,)]), action_signature=Signature(dtype=[dtype('float32')], shape=[(30,)]), reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), action_space=<ActionSpace.CONTINUOUS: 1>, action_size=30)
2025-05-18 18:40.15 [warning  ] Skip building models since they're already built.
2025-05-18 18:40.15 [info     ] Directory is created at training_logs/offline/relocate/IQL_20250518184015
2025-05-18 18:40.15 [info     ] Parameters                     params={'observation_shape': [39], 'action_size': 30, 'config': {'type': 'iql', 'params': {'batch_size': 256, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'compile_graph': False, 'actor_learning_rate': 0.0003, 'critic_learning_rate': 0.0003, 'actor_optim_factory': {'typ

Epoch 1/10:   0%|          | 0/100 [00:00<?, ?it/s]

2025-05-18 18:40.17 [info     ] IQL_20250518184015: epoch=1 step=100 epoch=1 metrics={'time_sample_batch': 0.003181352615356445, 'time_algorithm_update': 0.006346280574798584, 'critic_loss': 706.9986045837402, 'q_loss': 706.9981391906738, 'v_loss': 0.0004631680389138637, 'actor_loss': 10.649612110443414, 'time_step': 0.009580159187316894, 'environment': 5.188279189436387} step=100
2025-05-18 18:40.17 [info     ] Model parameters are saved to training_logs/offline/relocate/IQL_20250518184015/model_100.d3


Epoch 2/10:   0%|          | 0/100 [00:00<?, ?it/s]

2025-05-18 18:40.19 [info     ] IQL_20250518184015: epoch=2 step=200 epoch=2 metrics={'time_sample_batch': 0.001962294578552246, 'time_algorithm_update': 0.005901815891265869, 'critic_loss': 133.73566848754882, 'q_loss': 133.7096824645996, 'v_loss': 0.025986601340118796, 'actor_loss': -2.226599508523941, 'time_step': 0.007901437282562256, 'environment': 7.198446344109041} step=200
2025-05-18 18:40.19 [info     ] Model parameters are saved to training_logs/offline/relocate/IQL_20250518184015/model_200.d3


...

Epoch 9/10:   0%|          | 0/100 [00:00<?, ?it/s]

2025-05-18 18:41.53 [info     ] AWAC_20250518184132: epoch=9 step=900 epoch=9 metrics={'time_sample_batch': 0.0069163799285888675, 'time_algorithm_update': 0.010992271900177002, 'critic_loss': 198.96470092773438, 'actor_loss': 123277.99640625, 'temp': 0.0, 'temp_loss': 0.0, 'time_step': 0.017943065166473388, 'environment': 7.258779228649415} step=900
2025-05-18 18:41.53 [info     ] Model parameters are saved to training_logs/offline/relocate/AWAC_20250518184132/model_900.d3


Epoch 10/10:   0%|          | 0/100 [00:00<?, ?it/s]

2025-05-18 18:41.56 [info     ] AWAC_20250518184132: epoch=10 step=1000 epoch=10 metrics={'time_sample_batch': 0.006869642734527588, 'time_algorithm_update': 0.011153554916381836, 'critic_loss': 211.60583488464354, 'actor_loss': 109912.906953125, 'temp': 0.0, 'temp_loss': 0.0, 'time_step': 0.018059916496276855, 'environment': 6.972070138864593} step=1000
2025-05-18 18:41.56 [info     ] Model parameters are saved to training_logs/offline/relocate/AWAC_20250518184132/model_1000.d3


(<d3rlpy.algos.qlearning.awac.AWAC at 0x17abe2260>,
 [(1,
   {'time_sample_batch': 0.006716058254241943,
    'time_algorithm_update': 0.010795061588287353,
    'critic_loss': 1205.6437582397461,
    'actor_loss': 514241.23078125,
    'temp': 0.0,
    'temp_loss': 0.0,
    'time_step': 0.017547597885131837,
    'environment': 6.49402404980904}),
  (2,
   {'time_sample_batch': 0.007811052799224854,
    'time_algorithm_update': 0.014103903770446777,
    'critic_loss': 277.2829281616211,
    'actor_loss': 218237.0928125,
    'temp': 0.0,
    'temp_loss': 0.0,
    'time_step': 0.02196101188659668,
    'environment': 7.615295358374543}),
  (3,
   {'time_sample_batch': 0.006952996253967285,
    'time_algorithm_update': 0.01138843536376953,
    'critic_loss': 213.81610900878906,
    'actor_loss': 199744.95875,
    'temp': 0.0,
    'temp_loss': 0.0,
    'time_step': 0.018379724025726317,
    'environment': 8.653344223096}),
  (4,
   {'time_sample_batch': 0.00679694414138794,
    'time_algorithm

### Hammer

In [11]:
train_offline_algorithm(d3rlpy.algos.IQLConfig, hammer_d3_dataset, hammer_env, 'hammer_iql', 'hammer')
train_offline_algorithm(d3rlpy.algos.CQLConfig, hammer_d3_dataset, hammer_env, 'hammer_cql', 'hammer')
train_offline_algorithm(d3rlpy.algos.BCConfig, hammer_d3_dataset, hammer_env, 'hammer_bc', 'hammer')
train_offline_algorithm(d3rlpy.algos.TD3PlusBCConfig, hammer_d3_dataset, hammer_env, 'hammer_td3bc', 'hammer')
train_offline_algorithm(d3rlpy.algos.AWACConfig, hammer_d3_dataset, hammer_env, 'hammer_awac', 'hammer')

2025-05-18 18:41.56 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float64')], shape=[(46,)]), action_signature=Signature(dtype=[dtype('float32')], shape=[(26,)]), reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), action_space=<ActionSpace.CONTINUOUS: 1>, action_size=26)
2025-05-18 18:41.56 [warning  ] Skip building models since they're already built.
2025-05-18 18:41.56 [info     ] Directory is created at training_logs/offline/hammer/IQL_20250518184156
2025-05-18 18:41.56 [info     ] Parameters                     params={'observation_shape': [46], 'action_size': 26, 'config': {'type': 'iql', 'params': {'batch_size': 256, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'compile_graph': False, 'actor_learning_rate': 0.0003, 'critic_learning_rate': 0.0003, 'actor_optim_factory': {'type'

Epoch 1/10:   0%|          | 0/100 [00:00<?, ?it/s]

2025-05-18 18:41.58 [info     ] IQL_20250518184156: epoch=1 step=100 epoch=1 metrics={'time_sample_batch': 0.007340149879455566, 'time_algorithm_update': 0.006869847774505615, 'critic_loss': 8870.816901855469, 'q_loss': 8870.816279296874, 'v_loss': 0.0007186723929771688, 'actor_loss': 14.819753148555755, 'time_step': 0.01425492286682129, 'environment': -235.58140954462374} step=100
2025-05-18 18:41.58 [info     ] Model parameters are saved to training_logs/offline/hammer/IQL_20250518184156/model_100.d3


Epoch 2/10:   0%|          | 0/100 [00:00<?, ?it/s]

2025-05-18 18:41.59 [info     ] IQL_20250518184156: epoch=2 step=200 epoch=2 metrics={'time_sample_batch': 0.002027769088745117, 'time_algorithm_update': 0.007426340579986573, 'critic_loss': 2303.8857348632814, 'q_loss': 2303.7929418945314, 'v_loss': 0.092792750550434, 'actor_loss': 2.9383639964461326, 'time_step': 0.009489920139312744, 'environment': -231.18594692462443} step=200
2025-05-18 18:41.59 [info     ] Model parameters are saved to training_logs/offline/hammer/IQL_20250518184156/model_200.d3


...

Epoch 9/10:   0%|          | 0/100 [00:00<?, ?it/s]

2025-05-18 18:43.47 [info     ] AWAC_20250518184324: epoch=9 step=900 epoch=9 metrics={'time_sample_batch': 0.007092351913452148, 'time_algorithm_update': 0.011800620555877685, 'critic_loss': 1184.016176147461, 'actor_loss': 138509.465, 'temp': 0.0, 'temp_loss': 0.0, 'time_step': 0.0189325213432312, 'environment': -232.4682727325093} step=900
2025-05-18 18:43.47 [info     ] Model parameters are saved to training_logs/offline/hammer/AWAC_20250518184324/model_900.d3


Epoch 10/10:   0%|          | 0/100 [00:00<?, ?it/s]

2025-05-18 18:43.50 [info     ] AWAC_20250518184324: epoch=10 step=1000 epoch=10 metrics={'time_sample_batch': 0.007115712165832519, 'time_algorithm_update': 0.011782417297363281, 'critic_loss': 1199.7043359375, 'actor_loss': 133668.7353515625, 'temp': 0.0, 'temp_loss': 0.0, 'time_step': 0.018937323093414307, 'environment': -233.18178041608434} step=1000
2025-05-18 18:43.50 [info     ] Model parameters are saved to training_logs/offline/hammer/AWAC_20250518184324/model_1000.d3


(<d3rlpy.algos.qlearning.awac.AWAC at 0x17da76ef0>,
 [(1,
   {'time_sample_batch': 0.007343995571136475,
    'time_algorithm_update': 0.012756972312927247,
    'critic_loss': 4242.282707519531,
    'actor_loss': 471206.53640625,
    'temp': 0.0,
    'temp_loss': 0.0,
    'time_step': 0.02014333963394165,
    'environment': -236.434993416237}),
  (2,
   {'time_sample_batch': 0.006957356929779053,
    'time_algorithm_update': 0.01290010690689087,
    'critic_loss': 1757.700421142578,
    'actor_loss': 204663.432578125,
    'temp': 0.0,
    'temp_loss': 0.0,
    'time_step': 0.019899809360504152,
    'environment': -231.73257020335245}),
  (3,
   {'time_sample_batch': 0.00690997838973999,
    'time_algorithm_update': 0.011356468200683595,
    'critic_loss': 1308.7425939941406,
    'actor_loss': 184735.8736328125,
    'temp': 0.0,
    'temp_loss': 0.0,
    'time_step': 0.018306519985198975,
    'environment': -231.67110109072217}),
  (4,
   {'time_sample_batch': 0.006955242156982422,
    '

### Door

In [12]:
train_offline_algorithm(d3rlpy.algos.IQLConfig, door_d3_dataset, door_env, 'door_iql', 'door')
train_offline_algorithm(d3rlpy.algos.CQLConfig, door_d3_dataset, door_env, 'door_cql', 'door')
train_offline_algorithm(d3rlpy.algos.BCConfig, door_d3_dataset, door_env, 'door_bc', 'door')
train_offline_algorithm(d3rlpy.algos.TD3PlusBCConfig, door_d3_dataset, door_env, 'door_td3bc', 'door')
train_offline_algorithm(d3rlpy.algos.AWACConfig, door_d3_dataset, door_env, 'door_awac', 'door')

2025-05-18 18:43.50 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float64')], shape=[(39,)]), action_signature=Signature(dtype=[dtype('float32')], shape=[(28,)]), reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), action_space=<ActionSpace.CONTINUOUS: 1>, action_size=28)
2025-05-18 18:43.50 [warning  ] Skip building models since they're already built.
2025-05-18 18:43.50 [info     ] Directory is created at training_logs/offline/door/IQL_20250518184350
2025-05-18 18:43.50 [info     ] Parameters                     params={'observation_shape': [39], 'action_size': 28, 'config': {'type': 'iql', 'params': {'batch_size': 256, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'compile_graph': False, 'actor_learning_rate': 0.0003, 'critic_learning_rate': 0.0003, 'actor_optim_factory': {'type': 

Epoch 1/10:   0%|          | 0/100 [00:00<?, ?it/s]

2025-05-18 18:43.52 [info     ] IQL_20250518184350: epoch=1 step=100 epoch=1 metrics={'time_sample_batch': 0.005966944694519043, 'time_algorithm_update': 0.0074886465072631836, 'critic_loss': 229.86341054916383, 'q_loss': 229.86245990753173, 'v_loss': 0.000949906738824211, 'actor_loss': 16.686412696838378, 'time_step': 0.013509025573730469, 'environment': -44.061035208679314} step=100
2025-05-18 18:43.52 [info     ] Model parameters are saved to training_logs/offline/door/IQL_20250518184350/model_100.d3


Epoch 2/10:   0%|          | 0/100 [00:00<?, ?it/s]

2025-05-18 18:43.53 [info     ] IQL_20250518184350: epoch=2 step=200 epoch=2 metrics={'time_sample_batch': 0.0021971487998962403, 'time_algorithm_update': 0.006391475200653076, 'critic_loss': 8.263968176841736, 'q_loss': 8.2446111536026, 'v_loss': 0.019357012070249768, 'actor_loss': 1.9356118315458297, 'time_step': 0.00862600564956665, 'environment': 591.8126201373532} step=200
2025-05-18 18:43.53 [info     ] Model parameters are saved to training_logs/offline/door/IQL_20250518184350/model_200.d3


...

Epoch 9/10:   0%|          | 0/100 [00:00<?, ?it/s]

2025-05-18 18:45.24 [info     ] AWAC_20250518184501: epoch=9 step=900 epoch=9 metrics={'time_sample_batch': 0.0114914870262146, 'time_algorithm_update': 0.01214505434036255, 'critic_loss': 182.28158836364747, 'actor_loss': 225951.92875, 'temp': 0.0, 'temp_loss': 0.0, 'time_step': 0.02367981433868408, 'environment': -39.042494658037604} step=900
2025-05-18 18:45.24 [info     ] Model parameters are saved to training_logs/offline/door/AWAC_20250518184501/model_900.d3


Epoch 10/10:   0%|          | 0/100 [00:00<?, ?it/s]

2025-05-18 18:45.26 [info     ] AWAC_20250518184501: epoch=10 step=1000 epoch=10 metrics={'time_sample_batch': 0.007562184333801269, 'time_algorithm_update': 0.010577442646026612, 'critic_loss': 206.5586543273926, 'actor_loss': 205129.44234375, 'temp': 0.0, 'temp_loss': 0.0, 'time_step': 0.018180322647094727, 'environment': -39.69027961041189} step=1000
2025-05-18 18:45.26 [info     ] Model parameters are saved to training_logs/offline/door/AWAC_20250518184501/model_1000.d3


(<d3rlpy.algos.qlearning.awac.AWAC at 0x17dbc2140>,
 [(1,
   {'time_sample_batch': 0.007291326522827149,
    'time_algorithm_update': 0.011093316078186035,
    'critic_loss': 1337.3202682495116,
    'actor_loss': 688807.849375,
    'temp': 0.0,
    'temp_loss': 0.0,
    'time_step': 0.01842357397079468,
    'environment': -42.17264036625127}),
  (2,
   {'time_sample_batch': 0.00723168134689331,
    'time_algorithm_update': 0.011776893138885499,
    'critic_loss': 134.40468711853026,
    'actor_loss': 368667.305,
    'temp': 0.0,
    'temp_loss': 0.0,
    'time_step': 0.01904731035232544,
    'environment': -43.53126984787606}),
  (3,
   {'time_sample_batch': 0.007211339473724365,
    'time_algorithm_update': 0.011117961406707764,
    'critic_loss': 100.6105941772461,
    'actor_loss': 341866.265,
    'temp': 0.0,
    'temp_loss': 0.0,
    'time_step': 0.018370838165283204,
    'environment': -42.40779381954026}),
  (4,
   {'time_sample_batch': 0.007268142700195312,
    'time_algorithm_